## Introduction

import.py

In [1]:
from datetime import datetime
import time
import subprocess
import schedule
import mysql.connector

backup.py

In [ ]:
def backup_database(host, port, user, password, db_name):
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_file = f"backup_{timestamp}.sql"

    command = [
        "mysqldump", # <-- Change This Command If on Window
        f"--host={host}",
        f"--port={port}",
        f"--user={user}",
        f"--password={password}",
        db_name
    ]

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode == 0:
        with open(backup_file, 'wb') as f:
            f.write(result.stdout)
        print(f"Backup successful: {backup_file}")
    else:
        print(f"Backup failed: {result.stderr.decode('utf-8')}")

# Example
backup_database(
    host='maglev.proxy.rlwy.net',
    port=41425,
    user='root',    
    password='zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW',
    db_name='pos_management_db'
)

backup_scheduler.py

In [ ]:
def job():
    
    backup_database(
        "maglev.proxy.rlwy.net",
        41425,
        "root",
        "zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW",
        "pos_management_db"
    )

schedule.every(1).minute.do(job)

while True:
    schedule.run_pending()
    time.sleep(5)

recovery.py

In [ ]:
input_file="backup_20250711_042503.sql" # <-- Change With Your backup_timestamp.sql Date

def restore_database(host, port, user, password, db_name):
    command = [
        "mysql", # <-- Change This Command If on Window
        f"--host={host}",
        f"--port={port}",
        f"--user={user}"
    ]

    if password:
        command.append(f"--password={password}")

    command.append(db_name)
    
    mysql_conn = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password
    )
    cursor = mysql_conn.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{db_name}`")
    cursor.close()
    mysql_conn.close()

    with open(input_file, "r") as infile:
        result = subprocess.run(command, stdin=infile, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode == 0:
        print(f"Restore successful from {input_file}.")
    else:
        print(f"Error: {result.stderr.decode('utf-8')}")


# Example
restore_database(
    host='maglev.proxy.rlwy.net',
    port=41425,
    user='root',    
    password='zfZikcmJmTfiQOQKwToHWSJbaxBpmCpW',
    db_name='pos_management_recovery'
)